<a href="https://colab.research.google.com/github/RohanCheera/GEN-AI-Projects/blob/main/RAG_aplication_using_langchain_openai_and_faiss_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install langchain openai tiktoken rapidocr-onnxruntime

In [61]:

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY2')

In [62]:
pip install -U langchain-community

In [63]:
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from bs4 import BeautifulSoup
import lxml

In [64]:
url = "https://en.wikipedia.org/wiki/Mission_Bhagiratha"


In [65]:
res=requests.get(url)

In [66]:
res.text
soup=BeautifulSoup(res.text,'lxml')

In [67]:
#--Getting all the data with tag as 'p'
paras = [i.text for i in soup.find_all('p')]
print(paras)

['\n', 'Mission Bhagiratha is a safe drinking water project for every village in Telangana State, with a budget of ₹43,791 crores.[1] The project aims to provide piped water to 2.32 crore people in 20 lakh households in urban and 60 lakhs in rural areas of Telangana.[2] The project will supply clean drinking water to all households in the state through water sourced from the River Godavari (53.68 TMC) and the River Krishna (32.43TMC). The bulk supply is expected to be completed by May 2018, and intra-village, intra-locality works will be completed by December 2018.[3][needs update]\n', 'That project is named for King Bhagiratha, who, according to myth, brought the River Ganga to Earth from the heavens.\n', 'The inspiration for the project was driven by the residents who relied on contaminated groundwater. In drought-prone Nalgonda district, 973 villages faced high fluoride content that leads to disease and fluorosis.\n', 'A similar project, Maneru Manchineella Pathakam, was conceived a

In [68]:
doc="".join(paras)

In [69]:
doc=doc.replace('\n'," ")

In [70]:
with open("mission.txt","w") as f:
  f.write(doc)

In [71]:
loder=TextLoader('/content/mission.txt')

In [72]:
document=loder.load()
document[0].page_content

' Mission Bhagiratha is a safe drinking water project for every village in Telangana State, with a budget of ₹43,791 crores.[1] The project aims to provide piped water to 2.32 crore people in 20 lakh households in urban and 60 lakhs in rural areas of Telangana.[2] The project will supply clean drinking water to all households in the state through water sourced from the River Godavari (53.68 TMC) and the River Krishna (32.43TMC). The bulk supply is expected to be completed by May 2018, and intra-village, intra-locality works will be completed by December 2018.[3][needs update] That project is named for King Bhagiratha, who, according to myth, brought the River Ganga to Earth from the heavens. The inspiration for the project was driven by the residents who relied on contaminated groundwater. In drought-prone Nalgonda district, 973 villages faced high fluoride content that leads to disease and fluorosis. A similar project, Maneru Manchineella Pathakam, was conceived and completed by K.C.R

#**Chunking of the Data**

In [73]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [74]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [75]:
text_chunks=text_splitter.split_documents(document)

In [76]:
len(text_chunks)

9

In [77]:
print(text_chunks[3].page_content)

was completed successfully. It received an award from the Central Jal Shakti Minister Gajendra Singh Shekhawat in Rajya Sabha that Telangana is the only state in India providing piped water for every house hold in the state. The project was launched by the Prime Minister of India, Narendra Modi at Komatibanda village, Medak district in Gajwel constituency on 6 August 2016.[4] The other dignitaries were, the Chief Minister of Telangana, K Chandrashekar Rao. The central government-owned, Water


In [78]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [81]:
embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [82]:
!pip install faiss-cpu

In [83]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [91]:
retriever=vectorstore.as_retriever()

NameError: name 'vectorstore' is not defined

In [84]:
from langchain.prompts import ChatPromptTemplate

In [85]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [86]:
prompt=ChatPromptTemplate.from_template(template)

In [87]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [88]:
output_parser=StrOutputParser()

In [89]:
llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

In [90]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

NameError: name 'retriever' is not defined

In [ ]:
rag_chain.invoke("What is the mission of Mission Bhagiratha?")

In [ ]:
rag_chain.invoke("What is the purpose of Mission Bhagiratha?")